In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
class KMeans:
    def __init__(self, k: int, max_iter:int = 5):
        self.k = k
        self.max_iter = max_iter
        
    def initialize(self, X: np.ndarray):
        self.N, self.d = X.shape
        set = np.random.randint(0, self.N, self.k) # initial set
        self.centroids = np.array([X[i,:] for i in set])
        # self.sigma = np.array([ np.cov(X.T) for _ in range(self.k)])
        self.assignment = np.zeros(self.N)
        
    # def gaussian(x: np.ndarray, mu: np.ndarray, sigma: np.ndarray):
    #     idvs = len(x)
    #     norm_factor = (2*np.pi)**idvs

    #     norm_factor *= np.linalg.det(sigma)
    #     norm_factor = 1.0/np.sqrt(norm_factor)

    #     x_mu = np.matrix(x-mu)

    #     rs = norm_factor*np.exp(-0.5*x_mu*np.linalg.pinv(sigma)*x_mu.T)
    #     return rs
        
    def e_step(self,X):
        assert(self.N, self.d == X.shape)
        for i in range(self.N):
            x = X[i]
            scores = []
            for k in range(self.k):
                scores.append(np.norm(x - self.centroids[k]))
            self.assignment[i] = np.argmin(scores)
            
        
    
    def m_step(self,X):
        for k in range(self.k):
            ind_k = np.where(self.assignment == k)
            X_k = X[ind_k]
            self.centroids[k] = np.mean(X_k, axis=0)
            # self.sigma[k] = np.cov(X_k.T)

    def fit(self, X: np.ndarray):
        self.initialize(X)
        for _ in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)
            
    def predict(self,X: np.ndarray):
        if(len(X.shape) == 1):
            X = np.array([X])
        N,d = X.shape
        assignment = np.zeros(N)
        for i in range(N):
            x = X[i]
            scores = []
            for k in range(self.k):
                scores.append(KMeans.gaussian(x, self.centroids[k], self.sigma[k]))
            self.assignment[i] = np.argmax(scores)
        return assignment

<>:26: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:26: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\Vedant\AppData\Local\Temp\ipykernel_13952\925701895.py:26: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.N, self.d == X.shape)


In [14]:
from scipy.stats import multivariate_normal 
debug = False
class GMM:
    def __init__(self, k, max_iter=5):
        self.k = k
        self.max_iter = int(max_iter) 

    def initialize(self, X):
        # theta = [Mu, Sigma, Pi]
        n, d = X.shape
        set = np.random.randint(0, n, self.k) # initial set
        self.mu = np.array([X[i,:] for i in set])
        self.sigma = np.array([ np.cov(X.T) for _ in range(self.k) ])
        self.pi = np.ones(self.k)/self.k
        self.gamma = np.full((n,self.k),1/self.k)
        
        
        if(debug):
            print("-- INITIALIZATION GMM WITH k =", self.k, "--")
            print(self.mu)
            print(self.sigma)
            print(self.pi)

    def gaussian(x: np.ndarray, mu: np.ndarray, sigma: np.ndarray):
        idvs = len(x)
        norm_factor = (2*np.pi)**idvs

        norm_factor *= np.linalg.det(sigma)
        norm_factor = 1.0/np.sqrt(norm_factor)

        x_mu = np.matrix(x-mu)

        rs = norm_factor*np.exp(-0.5*x_mu*np.linalg.pinv(sigma)*x_mu.T)
        return rs

    # E-Step: 
    def e_step(self, X):
        n,d = X.shape
        for i in range(n):
            x = X[i]
            p_x = 0.
            vec = []
            for k in range(self.k):
                p_xk = self.pi[k] * (GMM.gaussian(x, self.mu[k], self.sigma[k]))
                vec.append(p_xk.item())
                p_x += p_xk
            self.gamma[i] = (np.array(vec))*1.0/p_x
        self.pi = np.mean(self.gamma, axis=0)

        # # updated weights or P(Xi/Ci=j)
        # self.weights = self.predict_proba(X)
        # # mean of sum of probability of all data points wrt to one cluster is new updated probability of cluster k or (phi)k
        # self.phi = self.weights.mean(axis=0)

    # M-Step: update meu and sigma holding phi and weights constant
    def m_step(self, X):
        # n,d = X.shape
        # for k in self.k:
        #     _mu_k_new = np.zeros(d);
        #     for i in range(n):
        #         _mu_k_new += self.gamma[i][k]*X[i]
        #     _mu_k_new /= self.gamma[:,k].sum()
        #     self.mu[k] = _mu_k_new
            # self.sigma[k] = np.cov(X.T,aweights=(weight/total_weight).flatten(), bias=True)
        for i in range(self.k):
            weight = self.gamma[:, [i]]
            total_weight = weight.sum()

            self.mu[i] = (X * weight).sum(axis=0) / total_weight
            self.sigma[i] = np.cov(X.T,aweights=(weight/total_weight).flatten(), bias=True)

    # responsible for clustering the data points correctly
    def fit(self, X):
        # initialise parameters like weights, phi, meu, sigma of all Gaussians in dataset X
        self.initialize(X)
        for iteration in range(self.max_iter):
            # iterate to update the value of P(Xi/Ci=j) and (phi)k
            self.e_step(X)
            # iterate to update the value of meu and sigma as the clusters shift
            self.m_step(X)
            

    # predicts probability of each data point wrt each cluster
    def predict_proba(self, X):
        # Creates a n*k matrix denoting probability of each point wrt each cluster 
        likelihood = np.zeros( (self.n, self.k) ) 
        for i in range(self.k):
            distribution = multivariate_normal(mean=self.mu[i],cov=self.sigma[i])
            # pdf : probability denisty function
            likelihood[:,i] = distribution.pdf(X) 

        numerator = likelihood * self.phi
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        weights = numerator / denominator
        return weights
    
    # predict function 
    def predict(self, X):
        n,d = np.array(X).shape
        gamma = np.zeros((n,self.k))
        for i in range(n):
            x = X[i]
            p_x = 0.
            vec = []
            for k in range(self.k):
                p_xk = self.pi[k] * (GMM.gaussian(x, self.mu[k], self.sigma[k]))
                vec.append(p_xk.item())
                p_x += p_xk
            gamma[i] = (np.array(vec))*1.0/p_x
        # self.pi = np.mean(self.gamma, axis=0)
        return np.argmax(gamma, axis=1)
        weights = self.predict_proba(X)
        # datapoint belongs to cluster with maximum probability
        # returns this value
        return np.argmax(weights, axis=1)
    
    def score(self, x, ret_gamma: bool = False):
        # x is d-dimensional single input feature vector
        if(ret_gamma):
            gamma = np.zeros(self.k)
            vec = []
        p_x = 0.
        for k in range(self.k):
            p_xk = self.pi[k] * (GMM.gaussian(x, self.mu[k], self.sigma[k]))
            if(ret_gamma):
                vec.append(p_xk.item())
            p_x += p_xk.item()
        if(ret_gamma):
            gamma = (np.array(vec))*1.0/p_x
            return p_x, gamma
        return p_x
    
    def plot_contour(self, fig, ax, _range, _input, color='red'):
        N, _, d = _input.shape
        if(d != 2):
            print("Can't plot for input not 2d")
            return None
        Z = np.empty((self.k,N,N))
        print("Plotting contours...")
        for k in tqdm(range(0,self.k)):
            for i in range(N):
                for j in range(N):
                    Z[k][i][j] = GMM.gaussian(_input[i][j], self.mu[k], self.sigma[k])
            ax.contour(_input[:,:,0], _input[:,:,1], Z[k], colors=color, levels=np.arange(0.2, 2.0, 0.5))
        print("Done")

In [4]:
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [5]:
import csv
X = []
Y = []
with open("6/train.txt") as f:
    reader = csv.reader(f)
    for row in reader:
        X.append([float(row[0]), float(row[1])])
        Y.append(int(row[2]))

df = pd.DataFrame(X, columns = ['x', 'y'])
df.head()

,x,y
0,-13.826,4.7990
1,-12.301,-1.3551
2,-13.968,4.3138
3,-13.898,5.3077
4,-12.690,5.9609


In [5]:
np.random.seed(42)
gmm = GMM(k=2, max_iter=20)
gmm.fit(np.array(X))

In [1]:
true = 0;
X_train = np.array(X)
Y_train = np.array(Y)
preds = gmm.predict(X)
for i in range(len(X_train)):
    x = X_train[i];
    y = Y_train[i];
    if(y == preds[i] + 1):
        true += 1
print(" Accuracy : ", true*100/len(X_train), "%")

NameError: name 'np' is not defined

In [6]:
Y = np.array(Y)
X = np.array(X)
ind1 = np.where(Y == 1)
print(ind1)
ind2 = np.where(Y == 2)
print(ind2)

X1 = X[ind1]
X2 = X[ind2]

(array([   0,    1,    2, ..., 1247, 1248, 1249], dtype=int64),)
(array([1250, 1251, 1252, ..., 2497, 2498, 2499], dtype=int64),)


In [19]:
gmm1 = GMM(32, 10)
gmm1.fit(X1)
gmm2 = GMM(32, 10)
gmm2.fit(X2)

In [8]:
import csv
X_dev = []
Y_dev = []
with open("6/dev.txt") as f:
    reader = csv.reader(f)
    for row in reader:
        X_dev.append([float(row[0]), float(row[1])])
        Y_dev.append(int(row[2]))



In [9]:
accuracy = 0.
for i in range(len(X_dev)):
    x = X_dev[i]
    s1 = gmm1.score(x)
    s2 = gmm2.score(x)
    if(s1 > s2):
        if(Y_dev[i] == 1):
            accuracy += 1
        else:
            accuracy -= 1
    elif(s2 > s1):
        if(Y_dev[i] == 2):
            accuracy += 1
        else:
            accuracy -= 1
    else:
        print("Ambigious point")
        # random
        r = np.random.randint(1,3)
        if(r == Y_dev[i]):
            accuracy += 1
        else:
            accuracy -= 1
accuracy *= 100/len(X_dev)
print(accuracy, "%")

100.0 %


In [10]:
post1 = lambda x: gmm1.score(x)
post2 = lambda x: gmm2.score(x)
classifier = lambda x : np.argmax([post1(x),post2(x)]) + 1

In [11]:
# construct a meshgrid using _range. N = no of sampling points(common for both x1, x2 intervals)
# returns the meshgrid(X1, X2) and Z = result of applying the 3 pdfs(pdf1 with mean mus[0], 
# covariance covs[0] and so on) on the meshgrid
def GridHelper(_range,N,mus,covs):
    X1 = np.linspace(_range[0][0], _range[0][1], N)
    X2 = np.linspace(_range[1][0], _range[1][1], N)
    X1, X2 = np.meshgrid(X1, X2)

    _input = np.empty((N,N,2))
    _input[:,:,0] = X1
    _input[:,:,1] = X2

    # Z = np.empty((2,N,N))
    # for i in range(0,2):
    #     Z[i] = GMM.gaussian(_input,mus[i],covs[i])
    return (X1,X2,_input)

In [17]:
# main plotting function
import matplotlib.colors as mcol
from tqdm.notebook import tqdm_notebook as tqdm
def DecisionBoundaryPlotter(_range,mus,covs,data,truth,classifier):

    fig, ax = plt.subplots(figsize=(10, 10))

    # construct grid and apply the three gaussians on it
    print("Making Grid...")
    X1, X2, _input = GridHelper(_range,100,mus,covs)
    print("Done.")

    # plot contours
    gmm1.plot_contour(fig,ax,_range,_input,'red')
    gmm2.plot_contour(fig,ax,_range,_input,'pink')

    # code to apply classifier function to each point on grid and use the result to plot 
    # decision regions
    _x1, _x2 = X1.flatten(), X2.flatten()
    surf = np.empty(_x1.shape)
    print("Evaluating classifier for ", _x1.size, " points...")
    for i in tqdm(range(0,_x1.size)):
        surf[i] = classifier([_x1[i],_x2[i]])
    print("Done.")
    surf = surf.reshape(X1.shape)
    ax.contourf(X1, X2, surf,cmap = mcol.ListedColormap(['orange', 'violet']))

    # code to plot the dev vectors labeled by their true class
    c1x1, c1x2 = [], []
    c2x1, c2x2 = [], []
    _sz = len(data)
    for i in range(0,_sz):
        if truth[i] == 1:
            c1x1.append(data[i][0]), c1x2.append(data[i][1])
        elif truth[i] == 2:
            c2x1.append(data[i][0]), c2x2.append(data[i][1])
    print("Plotting dev data points...")
    ax.scatter(c1x1, c1x2, edgecolors='yellow',label='Class 1 data',s=20,facecolors='none')
    ax.scatter(c2x1, c2x2, edgecolors='blue',label='Class 2 data',s=20,facecolors='none')
    print("Done.")

    # code to label axes and save the plot
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('Decision Boundary Diagram')
    ax.legend()
    plt.show()

Making Grid...
Done.
Plotting contours...


  0%|          | 0/32 [00:00<?, ?it/s]

Done
Plotting contours...


  0%|          | 0/32 [00:00<?, ?it/s]

Done
Evaluating classifier for  10000  points...


  0%|          | 0/10000 [00:00<?, ?it/s]

Done.
Plotting dev data points...
Done.


In [20]:
DecisionBoundaryPlotter([[-16,-2],[-7,8]], [gmm1.mu,gmm2.mu], [gmm1.sigma,gmm2.sigma], X_dev, Y_dev, classifier=classifier)